# [M_02] Protokół: Agent Strategii i Copywritingu (v2 Stability)

Ten moduł odpowiada za przekształcenie surowych danych analitycznych (z M_01) w kompletną strategię contentową oraz gotowe posty na platformy społecznościowe. 

**Wykorzystywany stos:**
* **Mózg:** Gemini 2.5 Flash (przez `pydantic-ai`).
* **Kontrakt:** Ścisłe modele Pydantic dla deterministycznych wyników JSON.
* **Nadzór:** Langfuse v2 (Self-hosted) – monitoring kosztów i logika agenta.

**Status Infrastruktury:** 
* Port 3000 (Langfuse) - AKTYWNY
* Port 6333 (Qdrant) - AKTYWNY

In [2]:
import os
import sys
import asyncio
from typing import List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Importy dla stabilnej wersji Langfuse v2.x i PydanticAI
from langfuse.decorators import observe, langfuse_context
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel

# 1. Zarządzanie ścieżkami (wyjście z notebooks/ do root projektu)
if os.getcwd().endswith("notebooks"):
    os.chdir("..")
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings
load_dotenv(".env")

# 2. Uzbrojenie środowiska (Standard v2)
os.environ["GOOGLE_API_KEY"] = settings.gemini_api_key
os.environ["LANGFUSE_PUBLIC_KEY"] = settings.langfuse_public_key
os.environ["LANGFUSE_SECRET_KEY"] = settings.langfuse_secret_key
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# 3. Inicjalizacja silnika Gemini
model = GoogleModel('gemini-2.5-flash')

print(f"STATUS: Jednostka gotowa. Monitoring Langfuse v2 aktywny.")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


STATUS: Jednostka gotowa. Monitoring Langfuse v2 aktywny.


## 1. Definicja Kontraktu Danych (Schema)

Zgodnie z protokołem "Code-First", definiujemy strukturę kampanii. Agent nie ma prawa zwrócić tekstu poza tym schematem, co gwarantuje poprawność integracji z przyszłymi modułami (np. publikatorem).

In [3]:
class PlatformPost(BaseModel):
    """Pojedyncza publikacja zoptymalizowana pod algorytm danej platformy."""
    platform: str = Field(description="TikTok, Instagram, YouTube lub LinkedIn")
    content: str = Field(description="Pełna treść posta wraz z hookiem i CTA")
    hashtags: List[str] = Field(description="Lista dopasowanych hashtagów")

class CampaignBrief(BaseModel):
    """Finalny produkt pracy Agenta Copywritera."""
    overall_strategy: str = Field(description="Główna linia narracyjna kampanii")
    posts: List[PlatformPost] = Field(description="Zestaw postów na różne kanały")

print("LOG: Modele danych zainicjalizowane.")

LOG: Modele danych zainicjalizowane.


## 2. Konfiguracja Agenta Operacyjnego

Inicjujemy Agenta z silnikiem Gemini 2.5 Flash. Wykorzystujemy parametr `output_type`, aby wymusić strukturę `CampaignBrief`.

In [4]:
# Inicjalizacja Agenta (Standard pydantic-ai 1.39.0)
copywriter_agent = Agent(
    model=model,
    output_type=CampaignBrief,
    system_prompt=(
        "Jesteś Szefem Strategii w KUŹNI OPERATORÓW. "
        "Twoim zadaniem jest tworzenie treści social media na podstawie danych technicznych. "
        "Dla każdego materiału przygotuj: "
        "1. TikTok: Agresywny hook, szybkie tempo, slang branżowy. "
        "2. YouTube: Skupienie na wartości edukacyjnej i SEO. "
        "3. LinkedIn: Merytoryczny wgląd, budowanie autorytetu, storytelling. "
        "Pisz wyłącznie po polsku."
    )
)
print("LOG: Agent Copywriter uzbrojony i gotowy do generowania.")

LOG: Agent Copywriter uzbrojony i gotowy do generowania.


## 3. Egzekucja i Monitoring (Observability)

W tej komórce uruchamiamy proces generowania strategii. Używamy dekoratora `@observe()`, który w wersji v2 automatycznie tworzy ślad (Trace) w Twoim lokalnym panelu.

In [5]:
# Przykładowe dane z Modułu 01
input_analysis_report = {
    "topic": "System OMNI-OPERATOR-V1 i suwerenność danych w AI",
    "main_points": ["Eliminacja SaaS", "Własny Stack", "Lokalny Qdrant"]
}

@observe(name="Run_Copywriter_Mission")
async def run_mission():
    try:
        print("LOG: Agent analizuje raport i tworzy kampanię...")
        
        # Wywołanie agenta (Wersja 1.39.0 zwraca ustrukturyzowany wynik w .output)
        result = await copywriter_agent.run(
            f"Przygotuj kampanię dla tych danych: {input_analysis_report}"
        )
        
        # Odbiór danych
        report = result.output
        
        print("\n" + "="*50)
        print(f"🚀 STRATEGIA: {report.overall_strategy}")
        print("="*50)
        
        for p in report.posts:
            print(f"\n📱 [{p.platform.upper()}]")
            print(f"📝 {p.content}")
            print(f"#️⃣ {', '.join(p.hashtags)}")
            
        print("\n" + "="*50)
        
        # Wymuszenie zapisu w Langfuse v2
        langfuse_context.flush()
        print("LOG: Dane przesłane do lokalnego Langfuse.")
        
        return report

    except Exception as e:
        print(f"❌ BŁĄD KRYTYCZNY: {str(e)}")

# URUCHOMIENIE
await run_mission()

LOG: Agent analizuje raport i tworzy kampanię...

🚀 STRATEGIA: Główna linia narracyjna kampanii skupia się na podkreślaniu kluczowej roli suwerenności danych w erze AI, pozycjonując KUŹNIĘ OPERATORÓW jako lidera innowacji poprzez system OMNI-OPERATOR-V1. Strategia ma na celu edukowanie rynku o ryzykach związanych z rozwiązaniami SaaS oraz promowanie niezależności i pełnej kontroli nad danymi poprzez własny stack technologiczny i lokalne rozwiązania takie jak Qdrant. Celem jest budowanie zaufania i autorytetu w dziedzinie bezpieczeństwa i przyszłości AI.

📱 [TIKTOK]
📝 Masz dość oddawania danych obcym? SaaS to przeżytek! 💥 Z OMNI-OPERATOR-V1 stawiasz własny stack. Koniec z uzależnieniem! Twoja AI, Twoje dane, 100% kontroli. Lokalny Qdrant to game changer. Suwerenność danych to podstawa. Nie daj się okraść z własności! #OMNIOPERATORV1 #AI #SuwerennośćDanych #SaaSKiller #Tech #Innowacje #Qdrant #DataPrivacy #OwnYourData
#️⃣ OMNIOPERATORV1, AI, SuwerennośćDanych, SaaSKiller, Tech, Innowacje

CampaignBrief(overall_strategy='Główna linia narracyjna kampanii skupia się na podkreślaniu kluczowej roli suwerenności danych w erze AI, pozycjonując KUŹNIĘ OPERATORÓW jako lidera innowacji poprzez system OMNI-OPERATOR-V1. Strategia ma na celu edukowanie rynku o ryzykach związanych z rozwiązaniami SaaS oraz promowanie niezależności i pełnej kontroli nad danymi poprzez własny stack technologiczny i lokalne rozwiązania takie jak Qdrant. Celem jest budowanie zaufania i autorytetu w dziedzinie bezpieczeństwa i przyszłości AI.', posts=[PlatformPost(platform='TikTok', content='Masz dość oddawania danych obcym? SaaS to przeżytek! 💥 Z OMNI-OPERATOR-V1 stawiasz własny stack. Koniec z uzależnieniem! Twoja AI, Twoje dane, 100% kontroli. Lokalny Qdrant to game changer. Suwerenność danych to podstawa. Nie daj się okraść z własności! #OMNIOPERATORV1 #AI #SuwerennośćDanych #SaaSKiller #Tech #Innowacje #Qdrant #DataPrivacy #OwnYourData', hashtags=['OMNIOPERATORV1', 'AI', 'SuwerennośćDanych', 'SaaSKil

## STATUS: MODUŁ 02 ZAKOŃCZONY

System posiada teraz w pełni funkcjonalny rurociąg komunikacyjny.

**Osiągnięto:**
1. Połączenie z Gemini 2.5 Flash przez PydanticAI.
2. Deterministyczne generowanie kampanii (Structured Outputs).
3. Stabilny monitoring przez Langfuse v2.

**Zadanie kontrolne:**
Wejdź na `http://localhost:3000`, znajdź Trace `Run_Copywriter_Mission` i sprawdź szczegóły wygenerowanego JSON-a.